1. models.py — Definiera modellerna (Customer, Account, Bank, Transaction)


In [ ]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey, DateTime
from sqlalchemy.orm import relationship, declarative_base
import datetime

Base = declarative_base()

class Bank(Base):
    __tablename__ = 'banks'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)

    accounts = relationship("Account", back_populates="bank")

class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    email = Column(String, unique=True)

    accounts = relationship("Account", back_populates="customer")

class Account(Base):
    __tablename__ = 'accounts'
    id = Column(Integer, primary_key=True)
    account_number = Column(String, unique=True)
    balance = Column(Float, default=0.0)

    customer_id = Column(Integer, ForeignKey('customers.id'))
    bank_id = Column(Integer, ForeignKey('banks.id'))

    customer = relationship("Customer", back_populates="accounts")
    bank = relationship("Bank", back_populates="accounts")
    transactions = relationship("Transaction", back_populates="account")

class Transaction(Base):
    __tablename__ = 'transactions'
    id = Column(Integer, primary_key=True)
    amount = Column(Float, nullable=False)
    timestamp = Column(DateTime, default=datetime.datetime.utcnow)

    account_id = Column(Integer, ForeignKey('accounts.id'))
    account = relationship("Account", back_populates="transactions")


2. Koppla models.py till Alembic i env.py

In [ ]:
from myproject.models import Base  # Ändra 'myproject' till  mapp
target_metadata = Base.metadata


3. Enkel testfil main.py för att testa att allt fungerar


In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Base, Bank, Customer, Account, Transaction

# Skapa databasanslutning (byt ut URL till din DB, här används SQLite för test)
engine = create_engine('sqlite:///bank.db')

# Skapa tabeller (om de inte finns)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Skapa testdata
bank = Bank(name="TestBank")
customer = Customer(name="Anna Andersson", email="anna@example.com")
account = Account(account_number="1234567890", balance=1000.0, customer=customer, bank=bank)
transaction = Transaction(amount=250.0, account=account)

# Lägg till och spara i databasen
session.add(bank)
session.add(customer)
session.add(account)
session.add(transaction)
session.commit()

# Hämta och skriv ut för att verifiera
for acc in session.query(Account).all():
    print(f"Account {acc.account_number} with balance {acc.balance} belongs to {acc.customer.name} at {acc.bank.name}")

session.close()
